### Call the load_ivy script.
Located in the source folder, the load_ivy.sc script will use ammonite to download snapshots of the chisel libraries, to be cached on your system. This is necessary for EVERY chisel notebook so we can then import our chisel libraries like we would in an SBT project structure.

In [1]:
//export
//default_exp test
import $file.source.load_ivy

import $file.$              

### Prevent namespace error
- To prevent an ambiguous reference to 'Input' by chisel3.Input and _root_.almond.input.Input, we have to make sure that we rename our chisel3 Input on import. It suffices to do as follows without requiring to change the name to something like IInput:
```
import chisel3.{Input => Input} 
```

In [2]:
//export
import chisel3.{Input => Input}
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import chisel3.tester.RawTester.test

import chisel3.{Input => Input}

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3.tester.RawTester.test

In Jupyter we are limitted to defining a class within a single cell. It can be argued that a notebook is excellent for hardware design because we can define modules and keep them well documented. However, Chisel modules often grow very large (i.e hundreds of lines). If this is required to be encapsulated in a single cell, then we will lose our ability to document what is happening within the module. Below is a workaround using Scala implicit conversions. 

In this example we will create a parameterizable "Operator" module that can perform either addition, subtraction, mul, or div. We will split up defining it over many cells and use implicit conversions to stitch it all together. 

### define the IO

In [3]:
//export 
class ALUIO extends Bundle {
    val a = Input(UInt(4.W))
    val b = Input(UInt(4.W))
    val out = Output(UInt(4.W))
}

defined class ALUIO

### a skeleton module will be parent class of all of our methods

In [4]:
//export 
class ALUSkeleton extends Module {
    val io = IO(new ALUIO)
}

defined class ALUSkeleton

### the add method from Add class will be injected into the ALUSkeleton class

In [5]:
//export 
class Add(m: ALUSkeleton) {
    def add(a: UInt, b: UInt): UInt = a + b
}

defined class Add

In [6]:
//export 
class Sub(m: ALUSkeleton) {
    def sub(a: UInt, b: UInt): UInt = a - b
}

defined class Sub

### Here multiple methods will be injected

In [7]:
//export 
class MulDiv(m: ALUSkeleton) {
    def mul(a: UInt, b: UInt): UInt = a * b
    def div(a: UInt, b: UInt): UInt = a / b
}

defined class MulDiv

### implicit def call will add the methods of the RHS classes to the implicit def parameter: ALUSkeleton class.
Note the name ```includeAdd``` does not matter as it won't be called again, but is good practice to give it a clear name of what it is doing (i.e includAdd <=> include the Add class' methods)

In [8]:
//export 
implicit def includeAdd(m: ALUSkeleton) = new Add(m)
implicit def includeSub(m: ALUSkeleton) = new Sub(m)
implicit def includeMulDiv(m: ALUSkeleton) = new MulDiv(m)

defined function includeAdd
defined function includeSub
defined function includeMulDiv

### Use our implicitly defined functions
We extend our ALUSkeleton giving us access to all of our implicitly defined methods. We can match based on parameterized operator (at compile time) and then only instantiate hardware for that given operation. 

In [9]:
//export
// This Operator module perform 1 type of operation depending on 'op' parameter
class Operator(op: String) extends ALUSkeleton {
    op match {
        // Call on the implicit function
        case "+" => io.out := this.add(io.a, io.b)
        case "-" => io.out := this.sub(io.a, io.b)
        case "*" => io.out := this.mul(io.a, io.b)
        case "/" => io.out := this.div(io.a, io.b)
        case _ => io.out := 0.U
    }
}

defined class Operator

### Summary
While this includes a few more steps than simply implementing everything directly into an Operator module, it can actually save some headaches when building much larger moudules. The best case for developing this way is if you're defining a specific arithmetic implmentation, which this example was meant to outline. For example a multiplier may require many sub methods to implement, and these implementation details can be hidden from the module using it. Also working like this puts you in the mindset of pulling out independent pieces of logic to write functions for that can later be reused in other modules. 

In [10]:
//export
test(new Operator(op="+")) { c =>
    c.io.a.poke(5.U); c.io.b.poke(3.U)
    c.io.out.expect(8.U)
    println(c.io.out.peek())
}


test(new Operator(op="-")) { c =>
    c.io.a.poke(5.U); c.io.b.poke(3.U)
    c.io.out.expect(2.U)
    println(c.io.out.peek())
}


test(new Operator(op="*")) { c =>
    c.io.a.poke(5.U); c.io.b.poke(3.U)
    c.io.out.expect(15.U)
    println(c.io.out.peek())
}


test(new Operator(op="/")) { c =>
    c.io.a.poke(5.U); c.io.b.poke(3.U)
    c.io.out.expect(1.U)
    println(c.io.out.peek())
}


test(new Operator(op="%")) { c =>
    c.io.a.poke(5.U); c.io.b.poke(3.U)
    c.io.out.expect(0.U)
    println(c.io.out.peek())
}

[info] [0.001] Elaborating design...
[info] [1.151] Done elaborating.
file loaded in 0.03387375 seconds, 7 symbols, 3 statements
UInt<4>(8)
test Operator Success: 0 tests passed in 2 cycles in 0.062334 seconds 32.09 Hz
[info] [0.000] Elaborating design...
[info] [0.159] Done elaborating.
file loaded in 0.016190125 seconds, 7 symbols, 3 statements
UInt<4>(2)
test Operator Success: 0 tests passed in 2 cycles in 0.039320 seconds 50.86 Hz
[info] [0.000] Elaborating design...
[info] [0.116] Done elaborating.
file loaded in 0.007054 seconds, 7 symbols, 3 statements
UInt<4>(15)
test Operator Success: 0 tests passed in 2 cycles in 0.034608 seconds 57.79 Hz
[info] [0.000] Elaborating design...
[info] [0.090] Done elaborating.
file loaded in 0.017142666 seconds, 6 symbols, 2 statements
UInt<4>(1)
test Operator Success: 0 tests passed in 2 cycles in 0.006663 seconds 300.14 Hz
[info] [0.001] Elaborating design...
[info] [0.092] Done elaborating.
file loaded in 0.005421125 seconds, 6 symbols, 2 sta

### Export the file to .sc
- File -> Export Notebook As... -> Executable Script
- Move into same directory as your other chisel notebooks
- This will create a file ToImport.scala
- Rename it to ToImport.sc
- Can now import using ```import $file.ToImport```

